1、使用FUNCTION_CALL模式

In [3]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent,AgentType,create_tool_calling_agent,AgentExecutor
from langchain.tools import Tool,StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()

#获取Tavily搜索工具的实例
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=3)


#获取一个搜索的工具
# search_tool = StructuredTool.from_function(
#     func=search.run,
#     name="Search",
#     description="用于检索互联网上的信息",
# )

search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


#获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')


#提供提示词模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手，根据用户的提问，必要时调用Search工具使用互联网检索数据"),
    ("human","{input}"),
    ("placeholder","{agent_scratchpad}")
])

#获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm = chat_model,
    tools = [search_tool],
    prompt = prompt_template,
)

#获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools = [search_tool],
    verbose=True
)

#通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `Search` with `今天北京天气情况`


[{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '| 气压 | 1012.5hPa | 1011.9hPa | 1010.1hPa | 1008.7hPa | 1009.2hPa | 1009.7hPa | 1009.4hPa | 1008.9hPa |\n| 湿度 | 93.2% | 73.3% | 79.1% | 78.5% | 84.3% | 87% | 90.4% | 92% |\n| 云量 | 0% | 26.6% | 10.1% | 45.4% | 78.5% | 62.5% | 90% | 0% | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 25.9℃ | 27.7℃ | 30.8℃ | 29.4℃ | 27.4℃ | 26℃ | 24.2℃ | 25.4℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 1.9mm |\n| 风速 | 3.2m/s | 3.2m/s | 3.3m/s | 2.5m/s | 2.6m/s | 2.6m/s | 2.4m/s | 2.8m/s |\n| 风向 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 东南风 | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 

使用ReAct模式

In [5]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent,AgentType,create_react_agent,AgentExecutor
from langchain.tools import Tool,StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()

#获取Tavily搜索工具的实例
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=3)


#获取一个搜索的工具
# search_tool = StructuredTool.from_function(
#     func=search.run,
#     name="Search",
#     description="用于检索互联网上的信息",
# )

search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


#获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')


template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

#提供提示词模板
prompt_template = PromptTemplate.from_template(template)

#获取Agent的实例：create_tool_calling_agent()
agent = create_react_agent(
    llm = chat_model,
    tools = [search_tool],
    prompt = prompt_template,
)

#获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools = [search_tool],
    verbose=True
)

#通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})
print(result)



> Entering new AgentExecutor chain...
我需要查询今天北京的天气情况。  
Action: Search  
Action Input: 今天北京天气  [{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 25.9℃ | 27.7℃ | 30.8℃ | 29.4℃ | 27.4℃ | 26℃ | 24.2℃ | 25.4℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 1.9mm |\n| 风速 | 3.2m/s | 3.2m/s | 3.3m/s | 2.5m/s | 2.6m/s | 2.6m/s | 2.4m/s | 2.8m/s |\n| 风向 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 东南风 | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 24.2℃ | 28.3℃ | 28.8℃ | 26.1℃ | 23.8℃ | 23.4℃ | 22.2℃ | 25.3℃ |\n| 降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 0.9mm | 无降水 | 0.7mm | 无降水 |\n| 风速 | 3.1m/s | 3.3m/s | 3.3m/s | 2.6m/s | 2.8m/s | 3m/s | 3.3m/s | 3.3m/s |\n| 风向 | 东北

In [6]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent,AgentType,create_react_agent,AgentExecutor
from langchain.tools import Tool,StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults
import langchain.hub as hub

dotenv.load_dotenv()

#获取Tavily搜索工具的实例
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=3)


#获取一个搜索的工具
# search_tool = StructuredTool.from_function(
#     func=search.run,
#     name="Search",
#     description="用于检索互联网上的信息",
# )

search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


#获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

#提供提示词模板
prompt_template = hub.pull("hwchase17/react")

#获取Agent的实例：create_tool_calling_agent()
agent = create_react_agent(
    llm = chat_model,
    tools = [search_tool],
    prompt = prompt_template,
)

#获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools = [search_tool],
    verbose=True
)

#通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})
print(result)



> Entering new AgentExecutor chain...
我需要查询今天北京的天气情况，可以通过网络搜索获取最新信息。  
Action: Search  
Action Input: 今天北京天气情况  [{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '| 气压 | 1012.5hPa | 1011.9hPa | 1010.1hPa | 1008.7hPa | 1009.2hPa | 1009.7hPa | 1009.4hPa | 1008.9hPa |\n| 湿度 | 93.2% | 73.3% | 79.1% | 78.5% | 84.3% | 87% | 90.4% | 92% |\n| 云量 | 0% | 26.6% | 10.1% | 45.4% | 78.5% | 62.5% | 90% | 0% | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 25.9℃ | 27.7℃ | 30.8℃ | 29.4℃ | 27.4℃ | 26℃ | 24.2℃ | 25.4℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 1.9mm |\n| 风速 | 3.2m/s | 3.2m/s | 3.3m/s | 2.5m/s | 2.6m/s | 2.6m/s | 2.4m/s | 2.8m/s |\n| 风向 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 东南风 | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n|

In [7]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent,AgentType,create_react_agent,AgentExecutor
from langchain.tools import Tool,StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()

#获取Tavily搜索工具的实例
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=3)


#获取一个搜索的工具
# search_tool = StructuredTool.from_function(
#     func=search.run,
#     name="Search",
#     description="用于检索互联网上的信息",
# )

search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


#获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')


system_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

ChatPromptTemplate.from_messages([
    ("system",system_template),
    ("human","{input}"),
    ("placeholder","{agent_scratchpad}")
])

#提供提示词模板
prompt_template = PromptTemplate.from_template(template)

#获取Agent的实例：create_tool_calling_agent()
agent = create_react_agent(
    llm = chat_model,
    tools = [search_tool],
    prompt = prompt_template,
)

#获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools = [search_tool],
    verbose=True
)

#通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})
print(result)



> Entering new AgentExecutor chain...
Thought: 我需要查询今天北京的天气情况，可以通过搜索引擎获取实时天气信息。
Action: Search
Action Input: 今天北京天气情况
Observ[{'title': '北京今晨有轻雾白天最高气温13℃ 明后天最低气温有所回升 - 资讯', 'url': 'http://news.weather.com.cn/2025/10/4376266.shtml', 'content': '大\n 中\n 小\n\n分享到\n\n原创未经授权不得转载\n\n中国天气网讯 今天（10月23日）早晨，北京将有轻雾，山区伴有雨夹雪或零星小雪，最高气温13℃，最低气温3℃。明后天，北京云量仍较多，最低气温有所上升，为5℃至6℃。\n\n昨天下午至夜间，北京雨水来扰，14时至22时全市平均降水量0.2毫米，城区微量，最大降水量出现在延庆竞速训练1站(高山站)，2.5毫米。\n\n今天早晨，北京天空云量较多。（图/中国天气网王晓）\n\n北京市气象台预计，今天早晨有轻雾，山区伴有雨夹雪或零星小雪，白天阴转多云，北转南风二三级，最高气温13℃；夜间多云，南转北风一二级，最低气温3℃。\n\n明后天，北京天空云量仍较多，最低气温稍微回升，为5℃至6℃。白天最高气温变化不大，维持在13℃上下。\n\n气象部门提醒，北京近期早晚气温较低，公众需注意及时添衣保暖，谨防呼吸道和心脑血管疾病。\n\n编辑：梁杰\n\n分享到\n\n查天气 关注 “中国天气网” 微信公众号、“中国天气”微博、“中国天气”抖音\n\n最新推荐\n\n查看更多\n\n高清图集 [...] 丰收色彩图鉴 世界粮食日看大地美学\n 河南汝州降雨路面湿滑 影响出行\n 爬山虎染秋色 浓烈似火\n\n原创策划\n\n天气视频\n\n 台风“风神”影响将逐渐减弱\n 注意！这波冷空气“电压不稳”！\n 专家解析：此轮冷空气接近尾声\n\n生活旅游\n\n 雨后峨眉沟壑尽显 金顶显真容\n 秋意浓 蓝天映衬下的哈尔滨伏尔加庄园\n 大美新疆—帕米尔高原好风光\n\n### 网站服务\n:   关于我们联系我们用户反馈\n\n    版权声明网站律师\n\n### 营销中心\n:   商务合作广告服务媒资合作\n\n### 相关链接